In [4]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import sys
sys.path.append('./src/')
import common_utils,my_ensembler, feature_utils, defines, model_utils, my_bert, classes
import numpy as np
import imp
import pandas as pd
import seaborn as sns
from sklearn_crfsuite import CRF
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, PassiveAggressiveClassifier, Perceptron, RidgeClassifier, RidgeClassifierCV, SGDClassifier
from sklearn.metrics import f1_score

In [5]:
dir_name='duplicate'
final_pred_name='all_pred_fixed.threshold0.256'#'merged.crf.bert.ens.c.w.l'
final_feature_name='local.plot.tfidf.30'
dataset_name='dataset_tf.word.char.lemma.full_splits'
cv_split_name='cv_splits_per_client'
seq_len=15
step=15
file_prefix='threshold'

In [6]:
all_pred_crf01=common_utils.load_db(dir_name,final_pred_name)

Opened /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/all_pred_fixed.threshold0.256.csv,  index False


In [7]:
score=f1_score(all_pred_crf01['ens_true'].tolist(),all_pred_crf01['ens_predicted'].tolist(),average='macro')

In [8]:
score

0.7309699060694252

In [8]:
cv_splits_loaded=common_utils.load_json(dir_name,cv_split_name)

Opened /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/cv_splits_per_client.json


In [4]:
db=all_pred_crf01

In [5]:
imp.reload(model_utils)
imp.reload(common_utils)
@interact
def show_metric_per_threshold(model='bert',threshold=(0.0,1.0,0.001)):
    th_db=pd.DataFrame()
    column='{}_proba_1'.format(model)
    cols=['{}_{}'.format(model,i) for i in ['split','true','predicted']]
    th_db=db[cols].copy()
    th_db['{}_predicted'.format(model)]  = db[column].apply(lambda y: 1 if y>=threshold else 0)
    sc,dict_,_,_=model_utils.get_report_by_unit(th_db,model,'split',n_t=2, print_rep=False, segeval =False, use_par=False)
    common_utils.plot_mean_scores(dict_)
    print('f1-score: {:.3f}'.format(np.mean(sc)))
    

interactive(children=(Text(value='bert', description='model'), FloatSlider(value=0.5, description='threshold',…

# Apply threshold to ensemble only

In [174]:
imp.reload(classes)
sgd_report=classes.MyScoreSummarizer(pred_df=all_pred_crf01,fix_list=['threshold_0.15','near_miss','single_miss','single_sa'],prefixes=['ens'])
sgd_report.get_all_scores(unit='split')
with pd.option_context('display.float_format', '{:0.3f}'.format):
    display(sgd_report.print_df['ens'])

MyScoreSummarizer init called
MyPredFixer init called
WindowDiff init called
Total pos 6501 ens before using threshold 0.15
Total pos 36801 ens after  using threshold 0.15
near misses to be fixed
 {'tot': 156, 'fp': 42, 'fn': 114}
1100 middle misses to be fixed
0 stande alone to be fixed
ens
label {0} is  missing in predicted for split 3, true is {0, 1}, pred is {1}
label {0} is  missing in predicted for split 4, true is {0, 1}, pred is {1}
label {0} is  missing in predicted for split 5, true is {0, 1}, pred is {1}
label {0} is  missing in predicted for split 7, true is {0, 1}, pred is {1}
MyReport init called


/home/zsofya/jupyter_git/jup-nb-generic/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofya/jupyter_git/jup-nb-generic/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofya/jupyter_git/jup-nb-generic/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.878,0.029,0.932,0.016,0.830,0.318
1,0.513,0.396,0.419,0.989,0.694,0.252
weighted avg,0.792,0.124,0.809,0.257,0.800,0.287


In [173]:
common_utils.save_db(sgd_report.print_df['ens'],dir_name,'ens.report.fixed.as.is.threshold=0.20')

Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.report.fixed.as.is.threshold=0.20.csv, 
index False
float_format %.5f


# Fix near miss, middle miss and stand alone after using threshold

In [49]:
smooth_report=classes.MyScoreSummarizer(pred_df=sgd_report.fixed_df,fix_list=['ner_miss','single_miss','single_sa'],prefixes=['ens'])
smooth_report.get_all_scores(unit='split')
with pd.option_context('display.float_format', '{:0.3f}'.format):
    display(smooth_report.print_df['ens'])

MyScoreSummarizer init called
MyPredFixer init called
WindowDiff init called
582 middle misses to be fixed
457 stande alone to be fixed
ens
MyReport init called


,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.861,0.865,0.858,0.860,0.866,0.871
1,0.591,0.604,0.595,0.608,0.596,0.612
weighted avg,0.799,0.804,0.798,0.804,0.802,0.809


In [145]:
common_utils.save_db(smooth_report.print_df['ens'],dir_name,'ens.report.as.is.threshold=0.16.smoothed')

Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.report.as.is.threshold=0.16.smoothed.csv, 
index False
float_format %.5f


In [144]:
smooth_report.print_df['ens']

,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.860990,0.864622,0.857533,0.860259,0.865897,0.870577
1,0.590515,0.603786,0.594762,0.608161,0.596164,0.612142
weighted avg,0.798570,0.804463,0.798062,0.803894,0.802347,0.808890


## Compare ens performance on 4 features & with threshold

# Fix threshold of CRF and BERT and then train ens on labels

## Fix threshold of CRF

In [52]:
imp.reload(classes)
model='crf'
crf_report=classes.MyScoreSummarizer(pred_df=all_pred_crf01,fix_list=['threshold_0.34'],prefixes=[model])
crf_report.get_all_scores(unit='split')
with pd.option_context('display.float_format', '{:0.3f}'.format):
    display(crf_report.print_df[model])

MyScoreSummarizer init called
MyPredFixer init called
WindowDiff init called
crf
Total pos 7990 crf before using threshold 0.34
Total pos 10929 crf after  using threshold 0.34
crf
MyReport init called


,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.863,0.848,0.895,0.838,0.834,0.860
1,0.511,0.564,0.454,0.581,0.611,0.559
weighted avg,0.782,0.783,0.793,0.782,0.783,0.789


In [59]:
common_utils.save_db(crf_report.print_df['crf'],dir_name,'crf.report.as.is.threshold=0.34')

Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/crf.report.as.is.threshold=0.34.csv, 
index False
float_format %.5f


## Fix threshold of BERT

In [54]:
imp.reload(classes)
model='bert'
bert_report=classes.MyScoreSummarizer(pred_df=all_pred_crf01,fix_list=['threshold_0.45'],prefixes=[model])
bert_report.get_all_scores(unit='split')
with pd.option_context('display.float_format', '{:0.3f}'.format):
    display(bert_report.print_df[model])

MyScoreSummarizer init called
MyPredFixer init called
WindowDiff init called
bert
Total pos 10796 bert before using threshold 0.45
Total pos 12366 bert after  using threshold 0.45
bert
MyReport init called


,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.829,0.811,0.818,0.778,0.841,0.847
1,0.517,0.518,0.539,0.584,0.500,0.469
weighted avg,0.756,0.743,0.753,0.734,0.761,0.758


In [60]:
common_utils.save_db(bert_report.print_df['bert'],dir_name,'bert.report.as.is.threshold=0.45')

Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/bert.report.as.is.threshold=0.45.csv, 
index False
float_format %.5f


## Combine BERT and CRF predictions fixed by threshold

In [55]:
fixed_th_df=all_pred_crf01.copy()
fixed_th_df['crf_predicted']=crf_report.fixed_df['crf_predicted']
fixed_th_df['bert_predicted']=bert_report.fixed_df['bert_predicted']
fixed_th_df[['crf_predicted','bert_predicted']].sum()

crf_predicted     10929
bert_predicted    12366
dtype: int64

## Train ensembled on 0,1 labels of CRF and BERT

In [56]:
estimators_pred_on_labels={}
imp.reload(feature_utils)
imp.reload(model_utils)

for regr in feature_utils.regressors_type:
    regr_name=regr.__class__.__name__
    print(regr_name)
    estimators_pred_on_labels[regr_name]=model_utils.prepared_cross_validate_ensemble(
        estimator=regr, 
        prediction_db_=fixed_th_df, 
        cv_splits=cv_splits_loaded,
        cols = [ 'crf_predicted', 'bert_predicted'])

LogisticRegression
0 split started...
1 split started...
2 split started...
3 split started...
4 split started...
5 split started...
6 split started...
7 split started...
LogisticRegressionCV
0 split started...
1 split started...
2 split started...
3 split started...
4 split started...
5 split started...
6 split started...
7 split started...
PassiveAggressiveClassifier
0 split started...
Cant't get probability for PassiveAggressiveClassifier
1 split started...
Cant't get probability for PassiveAggressiveClassifier
2 split started...
Cant't get probability for PassiveAggressiveClassifier
3 split started...
Cant't get probability for PassiveAggressiveClassifier
4 split started...
Cant't get probability for PassiveAggressiveClassifier
5 split started...
Cant't get probability for PassiveAggressiveClassifier
6 split started...
Cant't get probability for PassiveAggressiveClassifier
7 split started...
Cant't get probability for PassiveAggressiveClassifier
Perceptron
0 split started...
Cant't

In [63]:
# imp.reload(model_utils)
# crf_params={'seq_len':15,'seq_step':15}
# single_proba_estimators_pred['crf']=model_utils.prepared_cross_validate_ensemble(
#     estimator=CRF(), 
#     prediction_db_=fixed_th_df, 
#     cv_splits=cv_splits_loaded,
#     cols = [ 'crf_predicted', 'bert_predicted'],
#     **crf_params)

In [62]:
estimators_pred_on_labels['crf']=single_proba_estimators_pred['crf']

## Compare performance of different ensembles on 0,1 labels

In [157]:
labels_report

{'LogisticRegression': <classes.MyScoreSummarizer at 0x2b097f5f11f0>,
 'LogisticRegressionCV': <classes.MyScoreSummarizer at 0x2b0980130370>,
 'PassiveAggressiveClassifier': <classes.MyScoreSummarizer at 0x2b097f5f15e0>,
 'Perceptron': <classes.MyScoreSummarizer at 0x2b097a9fa490>,
 'RidgeClassifier': <classes.MyScoreSummarizer at 0x2b097f556af0>,
 'SGDClassifier': <classes.MyScoreSummarizer at 0x2b097f3d9ca0>,
 'SVC': <classes.MyScoreSummarizer at 0x2b0980610250>,
 'DecisionTreeClassifier': <classes.MyScoreSummarizer at 0x2b097fd93490>,
 'crf': <classes.MyScoreSummarizer at 0x2b097aa76fd0>}

In [159]:
labels_report={}
file_name='as.is'
for k,v in estimators_pred_on_labels.items():
    name=k
    labels_report[k]=classes.MyScoreSummarizer(pred_df=v,fix_list=[],prefixes=['ens'])
    labels_report[name].get_all_scores('split')
    labels_report[name].print_df['ens'].index.name='label'
    common_utils.save_db(v,dir_name,'ens.{}.pred.single.proba.{}'.format(name,file_prefix))
    common_utils.save_db(labels_report[name].print_df['ens'],dir_name,'ens.{}.report.single.proba.{}'.format(name,file_name))

MyScoreSummarizer init called
MyReport init called
Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.LogisticRegression.pred.single.proba.threshold.csv, 
index False
float_format %.5f
Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.LogisticRegression.report.single.proba.as.is.csv, 
index False
float_format %.5f
MyScoreSummarizer init called
MyReport init called
Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.LogisticRegressionCV.pred.single.proba.threshold.csv, 
index False
float_format %.5f
Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.LogisticRegressionCV.report.single.proba.as.is.csv, 
index False
float_format %.5f
MyScoreSummarizer init called
label {1} is  missing in predicted for split 0, true is {0, 1}, pred is {0}
label {1} is  missing in predicted for split 4, true is {0, 1}, 

/home/zsofya/jupyter_git/jup-nb-generic/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofya/jupyter_git/jup-nb-generic/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofya/jupyter_git/jup-nb-generic/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

label {1} is  missing in predicted for split 4, true is {0, 1}, pred is {0}
label {1} is  missing in predicted for split 7, true is {0, 1}, pred is {0}
MyReport init called
Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.Perceptron.pred.single.proba.threshold.csv, 
index False
float_format %.5f
Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.Perceptron.report.single.proba.as.is.csv, 
index False
float_format %.5f
MyScoreSummarizer init called


/home/zsofya/jupyter_git/jup-nb-generic/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofya/jupyter_git/jup-nb-generic/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofya/jupyter_git/jup-nb-generic/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

MyReport init called
Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.RidgeClassifier.pred.single.proba.threshold.csv, 
index False
float_format %.5f
Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.RidgeClassifier.report.single.proba.as.is.csv, 
index False
float_format %.5f
MyScoreSummarizer init called
MyReport init called
Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.SGDClassifier.pred.single.proba.threshold.csv, 
index False
float_format %.5f
Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.SGDClassifier.report.single.proba.as.is.csv, 
index False
float_format %.5f
MyScoreSummarizer init called
MyReport init called
Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.SVC.pred.single.proba.threshold.csv, 
index False
float_format %.5f
Sa

In [67]:
compare_scores=pd.DataFrame()
for k,v in labels_report.items():
    print(k)
    idx=v.print_df['ens'].index.tolist()
    compare_scores.loc[k,'common_f1']=v.print_df['ens'].loc['weighted avg','sent_f1']
    compare_scores.loc[k,'nar_recall']=v.print_df['ens'].loc[idx[1],'sent_recall']
    compare_scores.loc[k,'nar_prec']=v.print_df['ens'].loc[idx[1],'sent_prec']
compare_scores.sort_values(by='common_f1',ascending=False,inplace=True)

LogisticRegression
LogisticRegressionCV
PassiveAggressiveClassifier
Perceptron
RidgeClassifier
SGDClassifier
SVC
DecisionTreeClassifier
crf


In [68]:
compare_scores

,common_f1,nar_recall,nar_prec
crf,0.790320,0.464777,0.650613
SGDClassifier,0.783223,0.580609,0.558813
LogisticRegression,0.782045,0.403579,0.654659
LogisticRegressionCV,0.782045,0.403579,0.654659
RidgeClassifier,0.782045,0.403579,0.654659
SVC,0.782045,0.403579,0.654659
DecisionTreeClassifier,0.782045,0.403579,0.654659
PassiveAggressiveClassifier,0.755500,0.409488,0.446995
Perceptron,0.735724,0.356303,0.361084


## Fix prediction on ens trained on 0,1

In [162]:
imp.reload(classes)
labels_pred_crf_ens=common_utils.load_db(dir_name,'ens.crf.pred.single.proba.threshold')
labels_report['crf'].print_df['ens']

Opened /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.crf.pred.single.proba.threshold.csv,  index False


,sent_f1,sent_recall,sent_prec
label,,,
not_nar,0.869007,0.903593,0.838229
is_nar,0.530349,0.464777,0.650613
weighted avg,0.790320,0.801808,0.792852


In [166]:
imp.reload(classes)
crf_ens_fix=classes.MyScoreSummarizer(
    pred_df=labels_pred_crf_ens,
    fix_list=['threshold_0.30','near_miss','single_miss','single_sa'],
    prefixes=['ens'])
crf_ens_fix.get_all_scores()
crf_ens_fix.print_df['ens']

MyScoreSummarizer init called
MyPredFixer init called
WindowDiff init called
Total pos 7830 ens before using threshold 0.3
Total pos 11901 ens after  using threshold 0.3
near misses to be fixed
 {'tot': 38, 'fp': 20, 'fn': 18}
28 middle misses to be fixed
69 stande alone to be fixed
ens
MyReport init called


,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.869007,0.847553,0.903593,0.825768,0.838229,0.875544
1,0.530349,0.588254,0.464777,0.624398,0.650613,0.575175
weighted avg,0.790320,0.789054,0.801808,0.787128,0.792852,0.801131


In [168]:
common_utils.save_db(crf_ens_fix.print_df['ens'],dir_name,'ens.crf.fixed.report.single.proba.threshold')

Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.crf.fixed.report.single.proba.threshold.csv, 
index False
float_format %.5f


## Run ensemble only on 2 features

In [122]:
single_proba_estimators_pred={}
imp.reload(feature_utils)
imp.reload(model_utils)

for regr in feature_utils.regressors_type:
    regr_name=regr.__class__.__name__
    print(regr_name)
    single_proba_estimators_pred[regr_name]=model_utils.prepared_cross_validate_ensemble(
        estimator=regr, 
        prediction_db_=all_pred_crf01, 
        cv_splits=cv_splits_loaded,
        cols = [ 'crf_proba_1', 'bert_proba_1'])

LogisticRegression
0 split started...
1 split started...
2 split started...
3 split started...
4 split started...
5 split started...
6 split started...
7 split started...
LogisticRegressionCV
0 split started...
1 split started...
2 split started...
3 split started...
4 split started...
5 split started...
6 split started...
7 split started...
PassiveAggressiveClassifier
0 split started...
1 split started...
2 split started...
3 split started...
4 split started...
5 split started...
6 split started...
7 split started...
Perceptron
0 split started...
1 split started...
2 split started...
3 split started...
4 split started...
5 split started...
6 split started...
7 split started...
RidgeClassifier
0 split started...
1 split started...
2 split started...
3 split started...
4 split started...
5 split started...
6 split started...
7 split started...
SGDClassifier
0 split started...
1 split started...
2 split started...
3 split started...
4 split started...
5 split started...
6 split started..

# Check how to get probabilities from different estimators

In [110]:
ps=PassiveAggressiveClassifier()
X_train=np.random.rand(10,3)
X_test=np.random.rand(3,3)
y_train=np.random.randint(0,2,10)
X_train,X_test,y_train

(array([[0.91420638, 0.54690195, 0.05970407],
        [0.27377336, 0.13368256, 0.8497463 ],
        [0.40642918, 0.93865855, 0.42511328],
        [0.99964843, 0.29859993, 0.6455695 ],
        [0.94256683, 0.1372147 , 0.77275427],
        [0.41967425, 0.52983555, 0.83228755],
        [0.47651012, 0.39965027, 0.70490466],
        [0.68323192, 0.27621676, 0.90231308],
        [0.71947118, 0.87003106, 0.37479464],
        [0.53226444, 0.77597931, 0.55235245]]),
 array([[0.00342713, 0.17653862, 0.09324591],
        [0.11538997, 0.84462898, 0.23427792],
        [0.25780831, 0.41486092, 0.88257164]]),
 array([1, 0, 0, 1, 1, 1, 0, 0, 1, 0]))

In [111]:
ps.fit(X_train,y_train)

PassiveAggressiveClassifier()

In [115]:
ps._predict_proba_lr(X_test)

array([[0.88378499, 0.11621501],
       [0.92081425, 0.07918575],
       [0.96228163, 0.03771837]])

In [119]:
ps.decision_function(X_test)

array([-2.02877178, -2.45346196, -3.23915984])

In [120]:
probas={}
for regr in feature_utils.regressors_type:
    regr_name=regr.__class__.__name__
    print(regr_name)
    regr.fit(X_train,y_train)
    try:
        probas[regr_name]=regr.predict_proba(X_test)
    except:
        print('>'*10+' cant predict_proba')
        probas[regr_name]=regr._predict_proba_lr(X_test)

LogisticRegression
LogisticRegressionCV
PassiveAggressiveClassifier
>>>>>>>>>> cant predict_proba
Perceptron
>>>>>>>>>> cant predict_proba
RidgeClassifier
>>>>>>>>>> cant predict_proba
SGDClassifier
>>>>>>>>>> cant predict_proba
SVC
DecisionTreeClassifier


In [121]:
probas

{'LogisticRegression': array([[0.5652789 , 0.4347211 ],
        [0.56750078, 0.43249922],
        [0.58405739, 0.41594261]]),
 'LogisticRegressionCV': array([[0.89101946, 0.10898054],
        [0.86949176, 0.13050824],
        [0.88104319, 0.11895681]]),
 'PassiveAggressiveClassifier': array([[0.91382197, 0.08617803],
        [0.92497111, 0.07502889],
        [0.98418303, 0.01581697]]),
 'Perceptron': array([[0.54880205, 0.45119795],
        [0.55763738, 0.44236262],
        [0.79779132, 0.20220868]]),
 'RidgeClassifier': array([[0.59783733, 0.40216267],
        [0.59912301, 0.40087699],
        [0.61331918, 0.38668082]]),
 'SGDClassifier': array([[6.75349301e-01, 3.24650699e-01],
        [6.09859998e-03, 9.93901400e-01],
        [9.99931748e-01, 6.82522172e-05]]),
 'SVC': array([[0.53390867, 0.46609133],
        [0.35018512, 0.64981488],
        [0.29535727, 0.70464273]]),
 'DecisionTreeClassifier': array([[1., 0.],
        [1., 0.],
        [0., 1.]])}

In [123]:
imp.reload(model_utils)
crf_params={'seq_len':15,'seq_step':15}
single_proba_estimators_pred['crf']=model_utils.prepared_cross_validate_ensemble(
    estimator=CRF(), 
    prediction_db_=all_pred_crf01, 
    cv_splits=cv_splits_loaded,
    cols = [ 'crf_proba_1', 'bert_proba_1'],
    **crf_params)

0 split started...
1 split started...
2 split started...
3 split started...
4 split started...
5 split started...
6 split started...
7 split started...


In [124]:
single_proba_report={}
file_name='as.is'
for k,v in single_proba_estimators_pred.items():
    name=k
    single_proba_report[k]=classes.MyScoreSummarizer(pred_df=v,fix_list=[],prefixes=['ens'])
    single_proba_report[name].get_all_scores('split')
    single_proba_report[name].print_df['ens'].index.name='label'
    common_utils.save_db(v,dir_name,'ens.{}.pred.single.proba.as.is.{}'.format(name,file_prefix))
    common_utils.save_db(single_proba_report[name].print_df['ens'],dir_name,'ens.{}.report.single.proba.as.is{}'.format(name,file_name))

MyScoreSummarizer init called
MyReport init called
Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.LogisticRegression.pred.single.proba.as.is.threshold.csv, 
index False
float_format %.5f
Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.LogisticRegression.report.single.proba.as.isas.is.csv, 
index False
float_format %.5f
MyScoreSummarizer init called
MyReport init called
Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.LogisticRegressionCV.pred.single.proba.as.is.threshold.csv, 
index False
float_format %.5f
Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.LogisticRegressionCV.report.single.proba.as.isas.is.csv, 
index False
float_format %.5f
MyScoreSummarizer init called
label {1} is  missing in predicted for split 5, true is {0, 1}, pred is {0}
MyReport init called
Saving /scratch/sim_r

/home/zsofya/jupyter_git/jup-nb-generic/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofya/jupyter_git/jup-nb-generic/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofya/jupyter_git/jup-nb-generic/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.PassiveAggressiveClassifier.report.single.proba.as.isas.is.csv, 
index False
float_format %.5f
MyScoreSummarizer init called
label {1} is  missing in predicted for split 3, true is {0, 1}, pred is {0}
MyReport init called
Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.Perceptron.pred.single.proba.as.is.threshold.csv, 
index False
float_format %.5f


/home/zsofya/jupyter_git/jup-nb-generic/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofya/jupyter_git/jup-nb-generic/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofya/jupyter_git/jup-nb-generic/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.Perceptron.report.single.proba.as.isas.is.csv, 
index False
float_format %.5f
MyScoreSummarizer init called
MyReport init called
Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.RidgeClassifier.pred.single.proba.as.is.threshold.csv, 
index False
float_format %.5f
Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.RidgeClassifier.report.single.proba.as.isas.is.csv, 
index False
float_format %.5f
MyScoreSummarizer init called
MyReport init called
Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.SGDClassifier.pred.single.proba.as.is.threshold.csv, 
index False
float_format %.5f
Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.SGDClassifier.report.single.proba.as.isas.is.csv, 
index False
float_for

In [125]:
compare_scores=pd.DataFrame()
for k,v in single_proba_report.items():
    print(k)
    idx=v.print_df['ens'].index.tolist()
    compare_scores.loc[k,'common_f1']=v.print_df['ens'].loc['weighted avg','sent_f1']
    compare_scores.loc[k,'nar_recall']=v.print_df['ens'].loc[idx[1],'sent_recall']
    compare_scores.loc[k,'nar_prec']=v.print_df['ens'].loc[idx[1],'sent_prec']
compare_scores.sort_values(by='common_f1',ascending=False,inplace=True)

LogisticRegression
LogisticRegressionCV
PassiveAggressiveClassifier
Perceptron
RidgeClassifier
SGDClassifier
SVC
DecisionTreeClassifier
crf


In [126]:
compare_scores

,common_f1,nar_recall,nar_prec
SGDClassifier,0.793895,0.472800,0.648022
SVC,0.793434,0.422843,0.693999
RidgeClassifier,0.792263,0.429077,0.680229
LogisticRegression,0.791825,0.423378,0.686781
LogisticRegressionCV,0.791759,0.421753,0.689523
crf,0.789240,0.468709,0.645328
PassiveAggressiveClassifier,0.762559,0.482244,0.529350
Perceptron,0.741539,0.466937,0.565861
DecisionTreeClassifier,0.729699,0.443940,0.448526


## Fix best ens prediction on 2 labels

In [146]:
sgd_ens_pred_on_single=common_utils.load_db(dir_name,'ens.SGDClassifier.pred.single.proba.as.is.threshold')

Opened /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.SGDClassifier.pred.single.proba.as.is.threshold.csv,  index False


In [153]:
sgd_ens_pred_on_single[(sgd_ens_pred_on_single['ens_proba_0']>sgd_ens_pred_on_single['ens_proba_1'])&(sgd_ens_pred_on_single['ens_predicted']!=0)]

,ens_predicted,ens_proba_0,ens_proba_1,ens_group,ens_split,ens_true,ens_par


In [155]:
imp.reload(classes)
best_ens_fix=classes.MyScoreSummarizer(
    pred_df=sgd_ens_pred_on_single,
    fix_list=['threshold_0.30','near_miss','single_miss','single_sa'],
    prefixes=['ens'])
best_ens_fix.get_all_scores()
best_ens_fix.print_df['ens']

MyScoreSummarizer init called
MyPredFixer init called
WindowDiff init called
Total pos 7929 ens before using threshold 0.3
Total pos 15306 ens after  using threshold 0.3
near misses to be fixed
 {'tot': 117, 'fp': 45, 'fn': 72}
665 middle misses to be fixed
342 stande alone to be fixed
ens
MyReport init called


,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.871344,0.808323,0.905829,0.740862,0.840572,0.897076
1,0.536341,0.590912,0.472800,0.734387,0.648022,0.502900
weighted avg,0.793895,0.759907,0.804286,0.747886,0.796472,0.801494


In [81]:
compare_scores

,common_f1,nar_recall,nar_prec
SGDClassifier,0.793895,0.472800,0.648022
SVC,0.793434,0.422843,0.693999
RidgeClassifier,0.792263,0.429077,0.680229
LogisticRegression,0.791825,0.423378,0.686781
LogisticRegressionCV,0.791759,0.421753,0.689523
crf,0.789240,0.468709,0.645328
PassiveAggressiveClassifier,0.762559,0.482244,0.529350
Perceptron,0.741539,0.466937,0.565861
DecisionTreeClassifier,0.729699,0.443940,0.448526


In [25]:
sgd_double_proba=common_utils.load_db(dir_name,'ens.SGDClassifier.report.full.splits.c.w.l')
sgd_double_proba

Opened /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.SGDClassifier.report.full.splits.c.w.l.csv,  index False


,sent_f1,sent_recall,sent_prec
0,0.87031,0.91007,0.83617
1,0.52101,0.45387,0.65528
2,0.78853,0.80149,0.79550


# Fix predictions on single proba by best_ens

In [71]:
best_ens_pred=common_utils.load_db(dir_name,'ens.SGDClassifier.pred.full_split.c.w.l')

Opened /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.SGDClassifier.pred.full_split.c.w.l.csv,  index False


In [72]:
imp.reload(classes)
sgd_report=classes.MyScoreSummarizer(pred_df=best_ens_pred,fix_list=['threshold_0.16','single_miss','single_sa'],prefixes=['ens'])
sgd_report.get_all_scores(unit='split')

MyScoreSummarizer init called
MyPredFixer init called
WindowDiff init called
ens
No proba column for ens
309 middle misses to be fixed
138 stande alone to be fixed
ens
MyReport init called


In [73]:
sgd_report.print_df['ens']

,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.870308,0.871458,0.910069,0.908875,0.836174,0.839186
1,0.521007,0.532912,0.453874,0.467260,0.655279,0.661114
weighted avg,0.788533,0.792184,0.801489,0.803958,0.795498,0.798674


In [36]:
common_utils.save_db(sgd_report.print_df['ens'],dir_name,'ens.SGDClassifier.fixed.report.full.splits.c.w.l')

Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/ens.SGDClassifier.fixed.report.full.splits.c.w.l.csv, 
index False
float_format %.5f


## Fix ens predictions on threshold on 4 features